# Course Timetabling: Estudo de caso para o problema de agendamento de grade horária

## Fase 1: Alocação de professor em disciplina

Objetivo:
- Maximizar a performance acadêmica (EAP, professor habilitado em dar a disciplina) e o número de restrições fracas atendidas
- Alocar professores efetivos e substitutos em disciplinas (obrigatórias, eletivas e de serviço) conforme as regras do instituto da computação da UFRJ

### Modelagem

Conjuntos:

- $P$: Conjunto de professsores $p \in P$
- $PP$: Subconjunto de professores permanentes $PP \in P$
- $PS$: Subconjunto de professores substitutos $PS \in P$
- $D$: Conjunto de disciplinas ofertadas atrelada a uma turma T com o detalhamento da disciplina $d \in D$
- $Cred_d$: Conjunto de créditos das disciplinas $Cred_d \in D$
- $DAptas_p \subset D$: Subconjunto de disciplinas nas quais o professor p está habilitado a dar aulas.
- $H_d$: Subconjunto de horários das disciplinas $H_d \in D$
- $DS_d$: Subconjunto de dias da semana das disciplinas $DS_d \in D$
- $H_d$: Subconjunto com todos os horários possíveis para uma disciplina $H_d \in D$
- $DS_d$: Subconjunto com todos os dias da semana possíveis para uma disciplina $DS_d \in D$
- $D_h$: Subconjunto de disciplinas que possuem o horário h ($h \in H_d$)
- $D_{ds}$: Subconjunto de disciplinas que possuem o dia da semana ds ($ds \in DS_d$)



<!-- - $D_p \subset D$: Preferências de disciplinas de cada professor p -->
<!-- - $Tp \subset H Preferência de turno de cada professor p; -->

#### Variáveis

1) Alocação de professor em disciplina de uma turma 


$$
X_{p,d} \in (0,1) =

\begin{cases}
1 &  \quad \text{se o professor p for alocado na disciplina d}\\ 
0 & \quad \text{caso contrário} 
\end{cases}

$$

$$
p \in P, d \in D
$$

2. Variável de folga (slack variable) que indica quantos créditos o professor permanente está abaixo do ideal pela coordenação

$$
PNC_p \in \Z \quad p \in PP
$$ 


#### Coeficientes


1) Professor habilitado em dar disciplina recebe coeficiente 1

$$

f_{p,d} \in (0,1) = 

\begin{cases}
1 &  \quad d \in DAptas_p\\ 
0 & \quad \text{caso contrário}
\end{cases}

$$

$$
p \in P, d \in D \wedge	p \neq \text{'DUMMY'}
$$

2) Professor DUMMY é habilitado em todas as disciplinas. Ele recebe coeficiente 0.0001 pois só deve ser considerado se não existir outro professor apto.

$$

f_{p,d} = 0.0001 \quad d \in D, p \in P \wedge p = \text{'DUMMY'}

$$

3) Fator de penalidade no caso do professor não atingir a quantidade de créditos mínima

$$
    Wf = 1000
$$



### Função objetivo

Maximizar a performance acadêmica (EAP, professor habilitado em dar a disciplina) 
<!-- e o número de restrições fracas atendidas -->

$$
MAX \sum_{p \in P}\sum_{d \in D} f_{p,d}X_{p,d} - W_f \sum_{p \in PP} PNC_p
$$

### Restrições

### Restrições fracas

RF1: Garante que o professor seja alocado com a quantidade de créditos sujerida pela coordenação se possível. Não inviabilisa o modelo caso não seja atingido. 

Considera: 
- $FAL_{pp}$: Valor fixo que representa a quantidade de créditos que um professor deve ministrar. Número de créditos mínimos: 8. Cada professor precisa dar no mínimo 8 créditos horas de aula por semana (cada disciplina geralmente possui 4 créditos).


$$
\sum_{d \in D} cred_d X_{p,d} = FAL_{pp} - PNC_p \quad p \in PP \wedge	p \neq \text{'DUMMY'}
$$

##### Restrições fortes

RH2: Regime de trabalho (quantidade de horas) - quantidade de créditos máximo para o professor substituto

- $FAL_{ps}$: Número de créditos máximos: 12. Às vezes professores podem dar 12 créditos por semana. Para a equação, temos o professor p fixado e pertencente ao conjunto PS.

$$
\sum_{d \in D} cred_d X_{p,d} \leq FAL_{ps} \quad p \in PS \wedge	p \neq \text{'DUMMY'}
$$

RH3: Uma disciplina de uma turma, deverá ser ministrada por um único professor

$$
\sum_{p \in P} X_{p,d} = 1  \quad d \in D
$$

RH4: Um professor poderá dar no máximo 1 disciplina de uma turma em um mesmo dia e horário (binário OU >= 1)


$$
\sum_{d \in (D_h \cap D_{ds})} X_{p,d} \leq 1  \quad p \in P, h \in H_d, ds \in DS_d \wedge	p \neq \text{'DUMMY'}
$$

RH5: Um professor não pode lecionar uma disciplina em que ele não esteja apto

$$
\sum_{d \in (\Omega - DAptas_d)} X_{p,d} = 0 \quad p \in P \quad DAptas_d \in D
$$



